In [7]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports
import os

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\IEAP\\"
file_path = os.path.join(folder_path, "data.csv")

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR',)

In [8]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [9]:
dmm.write('*RST')
dmm.write(':DISPlay:VOLTage:DIGits 3')

27

### With Position

In [10]:
# 列を5つもつデータフレームに新しい行を追加する
def add_row(df, row):
    new_row = pd.DataFrame([row], columns=df.columns)
    return pd.concat([df, new_row], ignore_index=True)

In [11]:
#データフレーム、マルチメータ、合計測定時間、サンプル周期、印可電圧時間
def df_add_read_data(df,mul,total_read_time,sample_period):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()        
        mul.write("SENS:FUNC 'VOLT:DC', (@6)")
        mul.write("ROUT:CLOS (@6)")
        data1 = float(mul.query("READ?"))
        mul.write("SENS:FUNC 'VOLT:DC', (@7)")
        mul.write("ROUT:CLOS (@7)")
        data2 = float(mul.query("READ?"))
        mul.write("SENS:FUNC 'RES', (@8)")
        mul.write("ROUT:CLOS (@8)")
        data3 = float(mul.query("READ?"))
        mul.write("SENS:FUNC 'RES', (@9)")
        mul.write("ROUT:CLOS (@9)")
        data4 = float(mul.query("READ?"))
        mul.write("SENS:FUNC 'HERTz', (@10)")
        mul.write("ROUT:CLOS (@10)")
        data5 = float(mul.query("READ?"))
        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()
            
        df = add_row(df,[each_end_time - start_time,data1,data2,data3,data4,data5])
        end_time = time.perf_counter()
    return df

#### Measure

In [12]:
columns = ["Time[s]","Measured_Voltage[V]","Measured_Voltage[V]","Resistance[Ohm]","Resistance[Ohm]","Frequency[Hz]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq)
df_voltage

C:\Users\kary\AppData\Local\Temp\ipykernel_25260\2153073841.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, new_row], ignore_index=True)


,Time[s],Measured_Voltage[V],Measured_Voltage[V],Resistance[Ohm],Resistance[Ohm],Frequency[Hz]
0,0.348724,0.000002,0.081721,930.0201,9488.001,4796.841
1,0.605806,0.027291,0.155981,6431.3290,9537.947,4803.288
2,0.882733,0.004486,0.159395,6346.6250,9546.562,4805.235
3,1.160104,0.003022,0.161697,6341.3350,9550.249,4806.128
4,1.438140,0.002931,0.160607,6340.6550,9552.056,4806.578
5,1.716734,0.002900,0.160213,6340.3420,9553.026,4806.847
6,1.995914,0.002898,0.159653,6340.3540,9553.561,4807.006
7,2.275525,0.002887,0.158798,6340.2850,9553.898,4807.117
8,2.554072,0.002888,0.158727,6340.3940,9554.095,4807.188
9,2.833343,0.002915,0.158462,6340.4270,9554.240,4807.235
